In [ ]:
import os
import sys
import pandas as pd
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print('parent', parent_directory)
sys.path.append(parent_directory)
print(sys.path)

#### Read in the Reddit Data

In [ ]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(parent_directory)
directory = os.path.join(parent_directory,'data_collection/project_data/')
print(directory)
full_df = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".pkl"):
        file_path = os.path.join(directory, filename)
        df = pd.read_pickle(file_path)

        print(f"Data from {filename}:")
        #print(df.head())

        full_df = pd.concat([full_df, df], ignore_index=True)

print("Combined DataFrame:")
print(full_df)

In [ ]:
full_df

In [ ]:
full_df['Harm_Care_Agg'] = (full_df['HarmVice'] + full_df['HarmVirtue']) / 2
full_df['Authority_Agg'] = (full_df['AuthorityVice'] + full_df['HarmVirtue']) / 2
full_df['Purity_Agg'] = (full_df['PurityVice'] + full_df['PurityVirtue']) / 2
full_df['Fairness_Agg'] = (full_df['FairnessVice'] + full_df['FairnessVirtue']) / 2
full_df['Ingroup_Agg'] = (full_df['IngroupVice'] + full_df['IngroupVirtue']) / 2

In [ ]:
full_df.columns

In [ ]:

# Group by 'subreddit' and calculate the mean for each moral foundation
average_df = full_df.groupby('subreddit')[['HarmVirtue',
       'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice',
       'IngroupVice', 'FairnessVirtue',
       'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].mean().reset_index()

# Print the resulting DataFrame
print(average_df)


In [ ]:
full_df['Dominant_Moral_Foundation'] = full_df[['HarmVirtue', 'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice', 'IngroupVice', 'FairnessVirtue', 'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].idxmax(axis=1)

In [ ]:
full_df.sort_values(by = 'Harm_Care_Agg', ascending = False)

In [ ]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_df = full_df.groupby('subreddit')[['Harm_Care_Agg',
       'Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']].mean().reset_index()

# Print the resulting DataFrame
print(average_df)

In [ ]:
filtered_df = full_df[full_df['body'].str.len() > 50]

## i think this is the best one so far

In [ ]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_df = filtered_df.groupby('subreddit')[['HarmVirtue',
       'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice',
       'IngroupVice', 'FairnessVirtue',
       'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].mean().reset_index()

# Print the resulting DataFrame
print(average_df)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df = full_df

# Separate numeric and non-numeric columns
numeric_df = df.select_dtypes(include=['int64', 'float64'])
non_numeric_df = df.select_dtypes(exclude=['int64', 'float64'])

# Initialize and apply scaler to numeric columns
scaler = MinMaxScaler()
scaled_numeric_data = scaler.fit_transform(numeric_df)
scaled_numeric_df = pd.DataFrame(scaled_numeric_data, columns=numeric_df.columns)

# Concatenate scaled numeric columns back with non-numeric columns
final_scaled_df = pd.concat([scaled_numeric_df, non_numeric_df], axis=1)

# Display the final DataFrame
print(final_scaled_df)

In [ ]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_df = final_scaled_df.groupby('subreddit')[['HarmVirtue',
       'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice',
       'IngroupVice', 'FairnessVirtue',
       'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].mean().reset_index()

# Print the resulting DataFrame
print(average_df)

In [ ]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_df = final_scaled_df.groupby('subreddit')[['Harm_Care_Agg',
       'Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']].mean().reset_index()

# Print the resulting DataFrame
print(average_df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# Identifying the range of the aggregate scores to adjust the y-axis dynamically
min_score = average_df[['Harm_Care_Agg', 'Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']].min().min()
max_score = average_df[['Harm_Care_Agg', 'Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']].max().max()

# Adding a small buffer to min and max values for visual clarity
buffer = (max_score - min_score) * 0.05
y_min = max(0, min_score - buffer)  # Ensuring y_min doesn't go below 0
y_max = max_score + buffer

# Plotting with dynamic y-axis limits
average_df.set_index('subreddit').plot(kind='bar', figsize=(10, 6))
plt.title('Aggregate Moral Foundations Scores by Subreddit')
plt.ylabel('Aggregate Score')
plt.xlabel('Subreddit')
plt.xticks(rotation=45)
plt.ylim(y_min, y_max)
plt.legend(title='Moral Foundations')
plt.tight_layout()

plt.show()